In [2]:
#Modulo de llamadas http para descargar ficheros
!pip install requests

#Libreria del problema TSP: http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsplib.html
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1


In [3]:
import tsplib95
import random
from math import e
import urllib.request

In [4]:
#DATOS DEL PROBLEMA
file = "swiss42.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion(lista de nodos)
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


##Algoritmo de colonia de hormigas

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser mas eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$

In [ ]:
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )

# Nueva función para añadir un nodo basado en la probabilidad de transición definida por la ecuación (1)
# en el artículo de Dorigo y Gambardella (1997) (https://iridia.ulb.ac.be/~mdorigo/ACO/ACO.html) 
# con alpha=1 y beta=2 por defecto (valores recomendados por los autores) 
# y la feromona en la arista (i, j) elevada a alpha y la visibilidad de la arista (i, j) elevada a beta.
# La función devuelve el nodo seleccionado.
def Add_Nodo_Con_Funcion_Probabilidad(problem, H, T, alpha=1, beta=2):
    # Obtener la lista de nodos
    Nodos = list(problem.get_nodes())
    
    # Nodos disponibles (no visitados)
    nodos_disponibles = list(set(Nodos) - set(H))
    
    # Si no hay nodos disponibles, devolver None
    if not nodos_disponibles:
        return None
    
    # Calcular las probabilidades para cada nodo disponible
    probabilidades = []
    for nodo in nodos_disponibles:
        # Obtener el último nodo visitado
        ultimo_nodo = H[-1]
        
        # Calcular la visibilidad (inversa de la distancia)
        visibilidad = 1 / distancia(ultimo_nodo, nodo, problem)
        
        # Obtener la feromona en la arista
        feromona = T[ultimo_nodo - 1][nodo - 1]
        
        # Calcular el numerador de la probabilidad
        numerador = (feromona ** alpha) * (visibilidad ** beta)
        
        # Añadir el numerador a la lista de probabilidades
        probabilidades.append(numerador)
    
    # Calcular el denominador (suma de todos los numeradores)
    denominador = sum(probabilidades)
    
    # Calcular las probabilidades finales
    probabilidades_finales = [p / denominador for p in probabilidades]
    
    # Seleccionar el próximo nodo basado en las probabilidades
    siguiente_nodo = random.choices(nodos_disponibles, weights=probabilidades_finales, k=1)[0]
    
    return siguiente_nodo

def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [ ]:
def hormigas(problem, N) :
  #problem = datos del problema
  #N = Número de agentes(hormigas)

  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges())

  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]

  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]

  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :

      #Elige el siguiente nodo (usamos la nueva función)
      Nuevo_Nodo = Add_Nodo_Con_Funcion_Probabilidad(problem, Hormiga[h] ,T, alpha=1, beta=2 )
      Hormiga[h].append(Nuevo_Nodo)

    #Incrementa feromonas en esa arista
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)

    #Evapora Feromonas
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual


  print(mejor_solucion)
  print(mejor_distancia)


hormigas(problem, 1000)

[0, 1, 3, 27, 2, 28, 32, 10, 25, 41, 11, 12, 18, 5, 26, 8, 13, 19, 14, 37, 15, 16, 6, 4, 7, 35, 36, 17, 31, 20, 34, 30, 29, 22, 38, 9, 23, 40, 24, 21, 39, 33]
1999
